In [14]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
X_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
X_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [31]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Input ,concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers

In [32]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [60]:
CNN_model= Sequential()
CNN_model.add(Conv1D(filters=32, kernel_size=9, strides=2, activation='relu', input_shape=(n_timesteps,n_features)))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=128, kernel_size=1, strides=1, activation='relu'))
CNN_model.add(Flatten())
CNN_model.add(Dense(n_outputs, activation='relu'))
CNN_model.summary()

LSTM_model= Sequential()
LSTM_model.add(LSTM(64,input_shape=(n_timesteps,n_features),return_sequences=True))
LSTM_model.add(LSTM(64))
LSTM_model.add(Dropout(0.1))
LSTM_model.add(Dense(128, activation='relu'))
LSTM_model.add(Dense(n_outputs, activation='relu'))
LSTM_model.summary()

inputs = Input(shape=(n_timesteps,n_features))
mergedInput= concatenate([CNN_model(inputs),LSTM_model(inputs)])
out = Dense(n_outputs, activation='softmax')(mergedInput)
model3 = Model(inputs,out)
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()


CNN_model.trainable = False


Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_71 (Conv1D)          (None, 60, 32)            1760      
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 30, 32)           0         
 g1D)                                                            
                                                                 
 dropout_29 (Dropout)        (None, 30, 32)            0         
                                                                 
 conv1d_72 (Conv1D)          (None, 28, 64)            6208      
                                                                 
 conv1d_73 (Conv1D)          (None, 26, 128)           24704     
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 13, 128)          0         
 g1D)                                                

In [61]:
validation_data=(X_validation, y_validation)
history = model3.fit(X_training, y_training, epochs=10, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/10
207/207 [==============================] - 8s 25ms/step - loss: 2.4455 - accuracy: 0.3872 - val_loss: 0.9412 - val_accuracy: 0.7638
Epoch 2/10
207/207 [==============================] - 4s 21ms/step - loss: 0.8197 - accuracy: 0.7820 - val_loss: 0.5147 - val_accuracy: 0.8651
Epoch 3/10
207/207 [==============================] - 4s 21ms/step - loss: 0.5227 - accuracy: 0.8585 - val_loss: 0.4052 - val_accuracy: 0.8905
Epoch 4/10
207/207 [==============================] - 4s 21ms/step - loss: 0.4116 - accuracy: 0.8878 - val_loss: 0.3020 - val_accuracy: 0.9174
Epoch 5/10
207/207 [==============================] - 4s 21ms/step - loss: 0.3406 - accuracy: 0.9029 - val_loss: 0.3076 - val_accuracy: 0.9153
Epoch 6/10
207/207 [==============================] - 4s 21ms/step - loss: 0.2959 - accuracy: 0.9155 - val_loss: 0.2546 - val_accuracy: 0.9304
Epoch 7/10
207/207 [==============================] - 4s 21ms/step - loss: 0.2574 - accuracy: 0.9254 - val_loss: 0.2414 - val_accuracy: 0.9331

In [62]:
loss, accuracy = model3.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9391
Testing Accuracy: 0.8660
